# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [54]:
# Write your code below.

%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [55]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [57]:
import os
from glob import glob

# Write your code below.

PRICE_DATA = os.getenv("PRICE_DATA")
print(f"PRICE_DATA directory: {PRICE_DATA}")
parquet_files = glob(os.path.join(PRICE_DATA, "**", "**/*.parquet"), recursive = True)
print(f"Found parquet files: {parquet_files}")
dd_px = dd.read_parquet(parquet_files).set_index("Ticker")

PRICE_DATA directory: ../../05_src/data/prices/
Found parquet files: ['../../05_src/data/prices\\AAPL\\AAPL_2000\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2000\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2001\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2002\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2003\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2004\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2005\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.0.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2006\\part.1.parquet', '../../05_src/data/prices\\AAPL\\AAPL_2007\\part.0.parquet', '../../05_src/d

In [61]:
print(dd_px.head())

             Date       Close        High         Low        Open  \
Ticker                                                              
AAPL   2000-01-03    0.843076    0.847313    0.765877    0.789884   
AAPL   2025-01-02  243.850006  249.100006  241.820007  248.929993   
AAPL   2025-01-21  222.639999  224.419998  219.380005  224.000000   
AAPL   2025-01-17  229.979996  232.289993  228.479996  232.119995   
AAPL   2025-01-16  228.259995  238.009995  228.029999  237.350006   

             Volume  Year  
Ticker                     
AAPL    535796800.0  2000  
AAPL     55740700.0  2025  
AAPL     98070400.0  2025  
AAPL     68488300.0  2025  
AAPL     71759100.0  2025  


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [71]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(close_lag = x['Close'].shift(1))
)


C:\Users\joeyp\AppData\Local\Temp\ipykernel_13640\943940714.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [72]:
dd_ret = dd_shift.assign(
    returns = lambda x: x['Close']/x['close_lag'] - 1
)

In [73]:
dd_ret

,Date,Close,High,Low,Open,Volume,Year,close_lag,returns
npartitions=44,,,,,,,,,
AAPL,datetime64[ns],float64,float64,float64,float64,float64,int32,float64,float64
ACN,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
NVDA,...,...,...,...,...,...,...,...,...
NVDA,...,...,...,...,...,...,...,...,...


In [74]:
dd_ret.compute()

FileNotFoundError: [Errno 2] No such file or directory: 'c:/Users/joeyp/OneDrive/Documents/dsi/production/05_src/data/prices/ANSS/ANSS_2021/part.1.parquet'

In [ ]:
dd_shift.compute()

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [25]:
# Write your code below.



Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.